In [ ]:
###############################################################################
''''''
###############################################################################


import aliases

import numpy as np

from everest.ptolemaic import *
from everest.utilities.caching import soft_cache


class Space(System):

    name: Param
    dimensions: Param.Kwargs

    _pleroma_mroclasses__ = ('_Element_',)

    @classmethod
    def check_param(cls, arg, /):
        if isinstance(arg, Param):
            raise cls._BadParameter(arg)
        if arg is Param:
            raise cls._BadParameter(arg)
        return super().check_param(arg)

    class _Element_(System):

        hyperspace = None

        @classmethod
        def _cls_extra_init_(cls, /):
            super()._cls_extra_init_()
            if cls.hyperspace is None:
                return
            dimensions = cls.dimensions = cls.hyperspace.dimensions
            cls.contain_vectorized = tuple(
                np.vectorize(val.__contains__)
                for key, val in dimensions.items()
                )

        @classmethod
        def sub_sample(cls, sample, /):
            return sample[
                np.vstack(tuple(
                    vectorized(column)\
                    for vectorized, column
                    in zip(cls.contain_vectorized, sample.T)
                    ))
                .prod(axis=0).astype(bool)
                ]

        @classmethod
        def _ptolemaic_getitem__(cls, arg, /):
            if not isinstance(arg, tuple):
                arg = (arg,)
            space = cls.hyperspace
            newdims = space.dimensions.copy()
            for key, subarg in zip(newdims, arg):
                newdims[key] = newdims[key][subarg]
            return space.basecls(
                space.name,
                **newdims,
                )

    @property
    @soft_cache('_weakcache')
    def Element(self, /):
        dims = self.dimensions
        return self._Element_.subclass(
            _ptolemaic_annotations__={
                key: Param('PosKw', val)
                for key, val in dims.items()
                },
            name=f"{self.name}_{self.hashID}",
            dimensions=self.dimensions.copy(),
            hyperspace=self,
            )

    @classmethod
    def construct(cls, *args, **kwargs):
        space = super().construct(*args, **kwargs)
        return space.Element


###############################################################################
##############################################################################


In [ ]:
from everest.ptolemaic.sprites import Intt

In [ ]:
Foo = Space('Foo', a=Intt, b=Intt)

In [ ]:
SubClass = Foo[10:20, 30:40]

In [ ]:
SubClass

In [ ]:
Foo.classparams

In [ ]:
Foo(1, 2).params